## This notebook inputs the missing values using KNN

In [8]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.impute import KNNImputer
from sklearn import preprocessing

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [10]:
INPUT, OUTPUT = 'output/\CompleteCovid', 'output/\KNNCovidImputed'

In [18]:
df = pd.read_csv(r'output\/CompleteCovid.csv',index_col=0)
df_train = df.loc['2020-01-22 09:00:00':'2020-02-26 02:00:00'].dropna(how='all', axis=1) #Selecting training data
df_test = df.loc['2020-02-26 02:30:00':, df_train.columns] #Selecting test data
df = df_train

In [19]:
df

,Asia_confirmed_cases,Asia_deaths,Asia_recovered,Europe_confirmed_cases,Europe_deaths,Europe_recovered,Americas_confirmed_cases,Americas_deaths,Americas_recovered,Oceania_confirmed_cases,Oceania_deaths,Oceania_recovered,Africa_confirmed_cases,Africa_deaths,Africa_recovered
update_time,,,,,,,,,,,,,,,
2020-01-22 09:00:00,270.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-22 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-22 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-22 10:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-22 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-22 11:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-22 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-22 12:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-22 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
for i in range(0,30):
    #Scaled data has zero mean and unit variance
    scaler = preprocessing.StandardScaler()
    scaler.fit(df)
    array = scaler.transform(df)
    array_test = scaler.transform(df_test)
    df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
    df_test_scaled = pd.DataFrame(data=array_test, index=df_test.index, columns=df_test.columns)
    
    imp = KNNImputer(n_neighbors=5, weights="uniform")
    imp.fit(df_scaled)
    df_imputed = imp.transform(df_scaled)
    df_test_imputed = imp.transform(df_test_scaled)
    df_final = pd.DataFrame(data=df_imputed, index=df.index, columns=df.columns)
    df_test_final = pd.DataFrame(data=df_test_imputed, index=df_test.index, columns=df_test.columns)
    # Descaling the data
    array_2 = scaler.inverse_transform(df_final)
    df_final2 = pd.DataFrame(data=array_2, index=df.index, columns=df.columns)

    array_test_2 = scaler.inverse_transform(df_test_final)
    df_test_final2 = pd.DataFrame(data=array_test_2, index=df_test.index, columns=df_test.columns)
    df_final2.to_csv(OUTPUT+"_"+str(i)+'_training.csv', index = True)
    df_test_final2.to_csv(OUTPUT+"_"+str(i)+'_test.csv', index = True)